In [73]:
import pandas as pd
import numpy as np
import duckdb as db
import skimpy as sk
import chartify as ch

In [74]:
"""Issues Faced 
Loading Dataset (Unicode Error)
Data Capture Error
(Values stored in Customer_ID)
"""

'Issues Faced \nLoading Dataset (Unicode Error)\nData Capture Error\n(Values stored in Customer_ID)\n'

In [109]:
data = pd.read_csv("C:\\Users\\Admin\\Downloads\\Unclean_Data_1\\Unclean_Dataset_1.csv", encoding='ISO-8859-1')

data.tail()

,Student_ID,First_Name,Last_Name,Age,Gender,Course,Enrollment_Date,Total_Payments
131,232 | Amina | Adekunle | 21 | ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,234 | Ibrahim | Okafor | 22 | M...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,235 | Zainab | Abiodun | 23 | ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,236 | Nasiru | Adebayo | 24 | ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,237 | Aisha | Okonkwo | 25 | F...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
sk.skim(data)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 136    │ │ string      │ 8     │                                                          │
│ │ Number of columns │ 8      │ └─────────────┴───────┘                                                          │
│ └───────────────────┴────────┘                                                                                  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━┳━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓  │
│ ┃           ┃     ┃          ┃          ┃           ┃          ┃           ┃ chars    ┃ words per ┃ total    ┃  │
│ ┃ column    ┃ NA  ┃ NA %     ┃ shortest ┃ longest   ┃ min      ┃ max       ┃ per row  ┃ row       ┃ words    ┃  │
│ ┡━━━━━━━━━━━╇━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩  │
│ │ Student_I │   4 │ 2.941176 │ Lay      │ 149       │       |  │ Zainab    │       59 │        25 │     3459 │  │
│ │ D         │     │ 47058823 │          │ | Ava     │ Ben   |  │ Adeyemi   │          │           │          │  │
│ │           │     │       55 │          │ | Miller  │ Thompson │           │          │           │          │  │
│ │           │     │          │          │ | 25  | F │ | 20  |  │           │          │           │          │  │
│ │           │     │          │          │ | Machine │ M      | │           │          │           │          │  │
│ │           │     │          │          │ Learning  │ Web      │           │          │           │          │  │
│ │           │     │          │          │ |         │ Developm │           │          │           │          │  │
│ │           │     │          │          │ 2022-04-2 │ ent  |   │           │          │           │          │  │
│ │           │     │          │          │ 2      |  │ 2022-11- │           │          │           │          │  │
│ │           │     │          │          │ $1300     │ 14       │           │          │           │          │  │
│ │           │     │          │          │           │ | $1050  │           │          │           │          │  │
│ │  First_Na │  37 │ 27.20588 │ Lay      │ 149       │       |  │ Zainab    │     45.9 │        14 │     1943 │  │
│ │ me        │     │ 23529411 │          │ | Ava     │ Ben   |  │ Adeyemi   │          │           │          │  │
│ │           │     │       78 │          │ | Miller  │ Thompson │           │          │           │          │  │
│ │           │     │          │          │ | 25  | F │ | 20  |  │           │          │           │          │  │
│ │           │     │          │          │ | Machine │ M      | │           │          │           │          │  │
│ │           │     │          │          │ Learning  │ Web      │           │          │           │          │  │
│ │           │     │          │          │ |         │ Developm │           │          │           │          │  │
│ │           │     │          │          │ 2022-04-2 │ ent  |   │           │          │           │          │  │
│ │           │     │          │          │ 2      |  │ 2022-11- │           │          │           │          │  │
│ │           │     │          │          │ $1300     │ 14       │           │          │           │          │  │
│ │           │     │          │          │           │ 

In [77]:
data.columns = [x.strip().lower() for x in data.columns]

In [78]:
data = data.dropna(subset = 'student_id')

In [80]:
# 1. Extract rows with missing student_id
missing_student_id_df = data[data['student_id'].isnull()]

# 3. Append missing rows to uncleaned_data
uncleaned_data = pd.concat([uncleaned_data, missing_student_id_df], ignore_index=True)

# 4. Remove rows with missing student_id from main data
data = data.dropna(subset=['student_id'], how='any')

In [81]:
db.query("Select * from data where first_name is null and last_name is null and age is null and gender is null and course is null and enrollment_date is null and total_payments is null")

┌───────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬───────────┬─────────┬─────────┬─────────┬─────────────────┬────────────────┐
│                                            student_id                                             │ first_name │ last_name │   age   │ gender  │ course  │ enrollment_date │ total_payments │
│                                              varchar                                              │  varchar   │  varchar  │ varchar │ varchar │ varchar │     varchar     │    varchar     │
├───────────────────────────────────────────────────────────────────────────────────────────────────┼────────────┼───────────┼─────────┼─────────┼─────────┼─────────────────┼────────────────┤
│ 101        | John       | Smith     | 22  | M      | Data Science   | 2022-05-15      | $1200     │ NULL       │ NULL      │ NULL    │ NULL    │ NULL    │ NULL            │ NULL           │
│ 102        | Emily      | Johnson   | 

In [82]:
db.query("""Select * from data 
         where first_name is null 
         and last_name is null 
         and age is null 
         and gender is null 
         and course is null 
         and enrollment_date is null 
         and total_payments is null""")

┌───────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬───────────┬─────────┬─────────┬─────────┬─────────────────┬────────────────┐
│                                            student_id                                             │ first_name │ last_name │   age   │ gender  │ course  │ enrollment_date │ total_payments │
│                                              varchar                                              │  varchar   │  varchar  │ varchar │ varchar │ varchar │     varchar     │    varchar     │
├───────────────────────────────────────────────────────────────────────────────────────────────────┼────────────┼───────────┼─────────┼─────────┼─────────┼─────────────────┼────────────────┤
│ 101        | John       | Smith     | 22  | M      | Data Science   | 2022-05-15      | $1200     │ NULL       │ NULL      │ NULL    │ NULL    │ NULL    │ NULL            │ NULL           │
│ 102        | Emily      | Johnson   | 

In [83]:
data.columns

Index(['student_id', 'first_name', 'last_name', 'age', 'gender', 'course',
       'enrollment_date', 'total_payments'],
      dtype='object')

In [84]:
final_data = db.query("""Select 
         SPLIT_PART(student_id,'|',1) as student_id,
         SPLIT_PART(student_id,'|',2) as first_name,
         SPLIT_PART(student_id,'|',3) as last_name,
         SPLIT_PART(student_id,'|',4) as age,
         SPLIT_PART(student_id,'|',5) as gender,
         SPLIT_PART(student_id,'|',6) as course,
         SPLIT_PART(student_id,'|',7) as enrollment_date,
         SPLIT_PART(student_id,'|',8) as total_payments
         from data 
         where first_name is null 
         and last_name is null 
         and age is null 
         and gender is null 
         and course is null 
         and enrollment_date is null 
         and total_payments is null

""").df()

data = data.dropna(subset= ['first_name', 'last_name', 'age', 'gender', 'course','enrollment_date', 'total_payments'],how = 'all')

In [85]:
split_df = db.query("""
    SELECT 
        SPLIT_PART(student_id,'|',1) AS student_id,
        SPLIT_PART(student_id,'|',2) AS first_name,
        SPLIT_PART(student_id,'|',3) AS last_name,
        SPLIT_PART(student_id,'|',4) AS age,
        SPLIT_PART(student_id,'|',5) AS gender,
        SPLIT_PART(student_id,'|',6) AS course,
        SPLIT_PART(student_id,'|',7) AS enrollment_date,
        SPLIT_PART(student_id,'|',8) AS total_payments
    FROM data
    WHERE last_name IS NULL
      AND age IS NULL
      AND gender IS NULL
      AND course IS NULL
      AND enrollment_date IS NULL
      AND total_payments IS NULL
""").df()

final_data = pd.concat([final_data, split_df], ignore_index=True)

data = data.dropna(
    subset=['last_name', 'age', 'gender', 'course', 'enrollment_date', 'total_payments'],
    how='all'
)

In [86]:
age_gender_issue = db.query("""
SELECT 
         student_id,
         first_name,
         last_name,
    TRIM(LEFT(gender, 1)) AS gender,
    TRIM(SUBSTR(gender, 2)) AS age,
         course,
         enrollment_date
         ,total_payments
FROM data
WHERE gender IS NOT NULL and age is null
""").df()

final_data = pd.concat([final_data,age_gender_issue],ignore_index=True)

data = data.dropna(
subset = ['age'],how = 'all'
)


In [87]:
unkowns_replaced = db.query("""Select 
         student_id,first_name,COALESCE(last_name,'Unspecified') as last_name,age,gender,course,enrollment_date,total_payments 
         
          from data where last_name is null""").df()

final_data = pd.concat([final_data,unkowns_replaced],ignore_index=True)

data = data.dropna(subset = 'last_name',how='all')

In [88]:
data.isnull().sum()

student_id         0
first_name         0
last_name          0
age                0
gender             0
course             0
enrollment_date    0
total_payments     1
dtype: int64

In [89]:
unkowns = db.query("""
                   Select 
         student_id,first_name,last_name,age,gender,course,enrollment_date,total_payments 
         
          from data where last_name is null""").df()

final_data = pd.concat([final_data,unkowns],ignore_index=True)

data = data.dropna(subset = 'total_payments',how='all')

In [90]:
final_data = final_data.drop_duplicates()

In [91]:
final_data.loc[final_data['student_id'] == '', 'student_id'] = np.nan

In [92]:
# 1. Select all string (object) columns
str_cols = final_data.select_dtypes(include='object').columns

# 2. Strip leading/trailing whitespace
final_data[str_cols] = final_data[str_cols].apply(lambda x: x.str.strip())

# 3. Replace empty or whitespace-only strings with NaN
final_data[str_cols] = final_data[str_cols].replace(r'^\s*$', np.nan, regex=True)

In [93]:
final_data.loc[final_data['enrollment_date']=='NA','enrollment_date'] = np.nan

In [94]:
final_data['gender'] = final_data['gender'].astype(str).str.upper()

In [95]:
final_data.loc[final_data['course'].isin(['Web Developmen', 'Web Development', 'Web Develpment']), 'course'] = 'Web Development'
final_data.loc[final_data['course'].isin(['Machine Learnin', 'Machine Learning']), 'course'] = 'Machine Learning'
final_data.loc[final_data['course'] == '4', 'course'] = 'Course No.4'

In [96]:
final_data.loc[final_data['student_id'] == final_data['first_name'], 'student_id'] = 'Unknown'

In [97]:
con = db.connect(database=':memory:')
con.register('result', final_data)




In [98]:
final_data =con.query("""
WITH CTE AS (
    SELECT 
        student_id,
        first_name,
        last_name,
        age,
        gender,
        course,
        enrollment_date,
        TRIM(SUBSTR(total_payments, 2)) AS total_payments,  
        TRIM(LEFT(total_payments, 1)) AS Currency
    FROM result
)
SELECT
        student_id,
        first_name,
        last_name,
        age,
        gender,
        course,
        enrollment_date,

    TRIM(total_payments) AS total_payments,
    CASE 
        WHEN Currency = '?' THEN NULL
        ELSE Currency
    END AS Currency
      
          
FROM CTE

""").df()

In [99]:
con.query("Select enrollment_date  from result group by enrollment_date").df()

,enrollment_date
0,2022-03-18
1,2022-06-25
2,None
3,2022-08-15
4,2023-02-28
5,2022-11-14
6,2022-09-20
7,2022-07-05
8,2022-02-03
9,2022-05-09


In [ ]:
con.execute("""
UPDATE data
SET enrollment_date = CASE
    -- If format is dd-(alphabetic month)-yy, e.g., 01-Jan-21
    WHEN enrollment_date ~ '^\\d{2}-[A-Za-z]{3}-\\d{2}$' THEN
        STRFTIME(STRPTIME(enrollment_date, '%d-%b-%y'), '%d-%m-%Y')

    -- If format is dd-mm-yy numeric, e.g., 06-05-22
    WHEN enrollment_date ~ '^\\d{2}-\\d{2}-\\d{2}$' THEN
        STRFTIME(STRPTIME(enrollment_date, '%d-%m-%y'), '%d-%m-%Y')

    -- Already YYYY-MM-DD
    WHEN enrollment_date LIKE '20__%' THEN
        STRFTIME(STRPTIME(enrollment_date, '%Y-%m-%d'), '%d-%m-%Y')

    ELSE NULL
END
WHERE enrollment_date IS NOT NULL;
""")

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Admin\AppData\Local\Temp\ipykernel_1332\2043825622.py:5: SyntaxWarning: invalid escape sequence '\d'
  WHEN enrollment_date ~ '^\d{2}-[A-Za-z]{3}-\d{2}$' THEN


┌─────────────────┬──────────────────────────────┐
│ enrollment_date │ enrollment_date_standardized │
│     varchar     │           varchar            │
├─────────────────┼──────────────────────────────┤
│ 2022-05-15      │ 15-05-2022                   │
│ 2022-03-18      │ 18-03-2022                   │
│ 2022-06-25      │ 25-06-2022                   │
│ 2021-09-10      │ 10-09-2021                   │
│ 2022-04-22      │ 22-04-2022                   │
│ 2022-07-02      │ 02-07-2022                   │
│ 2022-08-15      │ 15-08-2022                   │
│ 2022-01-10      │ 10-01-2022                   │
│ 2023-02-28      │ 28-02-2023                   │
│ 2022-05-15      │ 15-05-2022                   │
│     ·           │     ·                        │
│     ·           │     ·                        │
│     ·           │     ·                        │
│ 21-Jul-24       │ 21-07-2024                   │
│ 30-Jan-21       │ 30-01-2021                   │
│ 30-Jan-20       │ 30-01-2020 

In [104]:
con.query("""
SELECT
    enrollment_date,
    STRFTIME(STRPTIME(enrollment_date, '%Y-%m-%d'), '%d-%m-%Y') AS enrollment_date_fixed
FROM result
WHERE enrollment_date LIKE '20__%'
""").df()

,enrollment_date,enrollment_date_fixed
0,2022-05-15,15-05-2022
1,2022-03-18,18-03-2022
2,2022-06-25,25-06-2022
3,2021-09-10,10-09-2021
4,2022-04-22,22-04-2022
...,...,...
62,2022-11-14,14-11-2022
63,2022-09-20,20-09-2022
64,2022-07-05,05-07-2022
65,2022-05-15,15-05-2022
